In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("HR_Dataset.csv")

In [ ]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Departments            14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [ ]:
df.salary = df.salary.map({"low":0,"medium":1,"high":2})

In [ ]:
X = df.drop(['left', "Departments "], axis=1)
y = df['left']

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop, Nadam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.losses import binary_crossentropy


In [ ]:

scaler = MinMaxScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=101)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
seed = 101
def build_classifier(optimizer, learn_rate):
    tf.random.set_seed(seed)
    classifier = Sequential()
    classifier.add(Dense(units=26, activation='relu'))
    classifier.add(Dense(units=13, activation='relu'))
    classifier.add(Dense(units=7, activation='relu'))
    classifier.add(Dense(units=1, activation='sigmoid'))
    classifier.compile(optimizer=optimizer(learn_rate),
                       loss='binary_crossentropy')
    return classifier

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min",
                           verbose=1, patience=5, restore_best_weights=True)

In [ ]:
classifier_model = KerasClassifier(
    build_fn=build_classifier, validation_split=0.1, epochs=200)

parameters = {'batch_size': [32, 64],
              'optimizer': [Adam, RMSprop],
              'learn_rate': [0.001, 0.003, 0.005]}

grid_model = GridSearchCV(estimator=classifier_model,
                          param_grid=parameters,
                          scoring='f1',
                          cv=2,
                          
                          verbose=1)

grid_model.fit(X_train, y_train, callbacks=[early_stop])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Fitting 2 folds for each of 12 candidates, totalling 24 fits
Epoch 1/200
169/169 [==============================] - 3s 4ms/step - loss: 0.5406 - val_loss: 0.4628
Epoch 2/200
169/169 [==============================] - 1s 3ms/step - loss: 0.4282 - val_loss: 0.4049
Epoch 3/200
169/169 [==============================] - 1s 3ms/step - loss: 0.3672 - val_loss: 0.3300
Epoch 4/200
169/169 [==============================] - 1s 3ms/step - loss: 0.2887 - val_loss: 0.2382
Epoch 5/200
169/169 [==============================] - 1s 3ms/step - loss: 0.2189 - val_loss: 0.1884
Epoch 6/200
169/169 [==============================] - 1s 3ms/step - loss: 0.1846 - val_loss: 0.1684
Epoch 7/200
169/169 [==============================] - 1s 3ms/step - loss: 0.1727 - val_loss: 0.1534
Epoch 8/200
169/169 [==============================] - 1s 3ms/step - loss: 0.1627 - val_loss: 0.1421
Epoch 9/200
169/169 [==============================] - 0s 3ms/step - loss: 0.1568 - val_loss: 0.1373
Epoch 10/200
169/169 [========

GridSearchCV(cv=2,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fef32f58050>,
             param_grid={'batch_size': [32, 64],
                         'learn_rate': [0.001, 0.003, 0.005],
                         'optimizer': [<class 'keras.optimizers.optimizer_v2.adam.Adam'>,
                                       <class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>]},
             scoring='f1', verbose=1)

In [ ]:
grid_model.best_params_

{'batch_size': 32,
 'learn_rate': 0.003,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam}

In [ ]:
grid_model.best_estimator_

In [ ]:
ypred = grid_model.predict(X_test)

94/94 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score

balanced_accuracy_score(y_test,ypred), accuracy_score(y_test,ypred)

(0.9482711719858548, 0.9666666666666667)

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.0 MB/s 
     |████████████████████████████████| 209 kB 66.3 MB/s 
     |████████████████████████████████| 81 kB 13.3 MB/s 
     |████████████████████████████████| 78 kB 10.1 MB/s 
     |████████████████████████████████| 112 kB 69.4 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 147 kB 70.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=12135524e160c6b17a6651b40ac765d21372a22ee46b0c5d7e932ad0cf1e0ce9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from tensorflow import keras

In [ ]:
from tensorflow.keras import *

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn import metrics
EPOCHS = 30 # number of epocs per trial
# BATCH_SIZE = 64

def objective(trial):
    
    # Clear clutter from previous session graphs.
    keras.backend.clear_session()
    
    # with tf.distribute.strategy.scope():
    # Generate our trial model.
    classifier = Sequential()
    activation = trial.suggest_categorical("activation", ["relu", "selu"])
    units_1 = trial.suggest_int("units_1",16,32,1)
    classifier.add(Dense(units=units_1, activation=activation))
    units_2 = trial.suggest_int("units_2",8,16,1)
    classifier.add(Dense(units=units_2, activation=activation))
    units_3 = trial.suggest_int("units_3",4,8,1)
    classifier.add(Dense(units=7, activation=activation))
    classifier.add(Dense(units=1, activation='sigmoid'))
    optimizer = trial.suggest_categorical("optimizer", [Adam, RMSprop])
    learn_rate = trial.suggest_float("learn_rate",0.0,0.3)
    classifier.compile(optimizer=optimizer(learn_rate),
                       loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall()])
   
    # Fit the model on the training data.
    # The TFKerasPruningCallback checks for pruning condition every epoch.
    BATCH_SIZE = trial.suggest_categorical("BATCH_SIZE", [32, 64])
    classifier.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        epochs=EPOCHS,
        validation_split = .15,  #validation_data=(X_test, y_test),
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    score = classifier.evaluate(X_test, y_test, verbose=1)
    return score[1]

In [ ]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=10)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2022-10-28 07:57:55,105] A new study created in memory with name: no-name-6282144f-0cb8-484a-943e-b2ddb755acf8


Epoch 1/30


/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'keras.optimizers.optimizer_v2.adam.Adam'> which is of type type.
  warnings.warn(message)
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'> which is of type type.
  warnings.warn(message)


319/319 [==============================] - 3s 5ms/step - loss: 0.5545 - recall: 0.0058 - val_loss: 0.5614 - val_recall: 0.0000e+00
Epoch 2/30
319/319 [==============================] - 1s 5ms/step - loss: 0.5487 - recall: 0.0000e+00 - val_loss: 0.5720 - val_recall: 0.0000e+00
Epoch 3/30
319/319 [==============================] - 2s 7ms/step - loss: 0.5495 - recall: 0.0000e+00 - val_loss: 0.5657 - val_recall: 0.0000e+00
Epoch 4/30
319/319 [==============================] - 3s 8ms/step - loss: 0.5494 - recall: 0.0000e+00 - val_loss: 0.5732 - val_recall: 0.0000e+00
Epoch 5/30
319/319 [==============================] - 3s 8ms/step - loss: 0.5492 - recall: 0.0000e+00 - val_loss: 0.5621 - val_recall: 0.0000e+00
Epoch 6/30
319/319 [==============================] - 2s 7ms/step - loss: 0.5497 - recall: 0.0000e+00 - val_loss: 0.5628 - val_recall: 0.0000e+00
Epoch 7/30
319/319 [==============================] - 2s 7ms/step - loss: 0.5496 - recall: 0.0000e+00 - val_loss: 0.5667 - val_recall: 0.00

[I 2022-10-28 07:59:18,616] Trial 0 finished with value: 0.0 and parameters: {'activation': 'relu', 'units_1': 16, 'units_2': 14, 'units_3': 7, 'optimizer': <class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, 'learn_rate': 0.13745633830485332, 'BATCH_SIZE': 32}. Best is trial 0 with value: 0.0.


...

[I 2022-10-28 08:02:25,635] Trial 7 pruned. Trial was pruned at epoch 3.


Epoch 1/30
319/319 [==============================] - 2s 4ms/step - loss: 0.3525 - recall: 0.6048 - val_loss: 0.2034 - val_recall: 0.8371
Epoch 2/30
319/319 [==============================] - 1s 4ms/step - loss: 0.2196 - recall: 0.8364 - val_loss: 0.3282 - val_recall: 0.4174
Epoch 3/30
319/319 [==============================] - 1s 4ms/step - loss: 0.2121 - recall: 0.8543 - val_loss: 0.2034 - val_recall: 0.8683
Epoch 4/30
319/319 [==============================] - 1s 4ms/step - loss: 0.1932 - recall: 0.8622 - val_loss: 0.2197 - val_recall: 0.9107
Epoch 5/30
319/319 [==============================] - 1s 4ms/step - loss: 0.2097 - recall: 0.8543 - val_loss: 0.1720 - val_recall: 0.8705
Epoch 6/30
319/319 [==============================] - 1s 4ms/step - loss: 0.2042 - recall: 0.8713 - val_loss: 0.2354 - val_recall: 0.9219
Epoch 7/30
319/319 [==============================] - 1s 4ms/step - loss: 0.1822 - recall: 0.8763 - val_loss: 0.1957 - val_recall: 0.7991
Epoch 8/30
319/319 [==============

[I 2022-10-28 08:02:38,920] Trial 8 pruned. Trial was pruned at epoch 9.


Epoch 1/30
160/160 [==============================] - 2s 6ms/step - loss: 0.6483 - recall: 0.1540 - val_loss: 0.5006 - val_recall: 0.9420
Epoch 2/30
160/160 [==============================] - ETA: 0s - loss: 0.3431 - recall: 0.7721

[I 2022-10-28 08:02:49,602] Trial 9 pruned. Trial was pruned at epoch 1.


In [ ]:
study.best_trial.params


{'activation': 'selu',
 'units_1': 21,
 'units_2': 15,
 'units_3': 8,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam,
 'learn_rate': 0.10319618640075823,
 'BATCH_SIZE': 64}

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_param_importances(study)

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  5
  Number of complete trials:  5
Best trial:
  Value:  0.9299719929695129
  Params: 
    activation: selu
    units_1: 21
    units_2: 15
    units_3: 8
    optimizer: <class 'keras.optimizers.optimizer_v2.adam.Adam'>
    learn_rate: 0.10319618640075823
    BATCH_SIZE: 64
